In [1]:
from pyspark import SparkContext
sc = SparkContext(appName = "lab1-spark")
temperature_file = sc.textFile("input_data/temperature-readings.csv")
lines = temperature_file.map(lambda line: line.split(";"))

In [2]:
# 1
year_temperature = lines.map(lambda x: (x[1][0:4], float(x[3])))
year_temperature = year_temperature.filter(lambda x: int(x[0])>=1950 and int(x[0])<=2014)

max_temperatures = year_temperature.reduceByKey(max)
min_temperatures = year_temperature.reduceByKey(min)

max_temperatures = max_temperatures.sortBy(ascending = False, keyfunc=lambda k: k[1])
min_temperatures = min_temperatures.sortBy(ascending = False, keyfunc=lambda k: k[1])

print(min_temperatures.collect())
print(max_temperatures.collect())

[('1990', -35.0), ('1952', -35.5), ('1974', -35.6), ('1954', -36.0), ('1992', -36.1), ('1975', -37.0), ('1972', -37.5), ('2000', -37.6), ('1995', -37.6), ('1957', -37.8), ('1983', -38.2), ('1989', -38.2), ('1953', -38.4), ('2009', -38.5), ('1993', -39.0), ('1984', -39.2), ('1991', -39.3), ('1973', -39.3), ('2008', -39.3), ('2005', -39.4), ('1961', -39.5), ('1964', -39.5), ('1970', -39.6), ('2004', -39.7), ('1988', -39.9), ('1960', -40.0), ('1997', -40.2), ('1994', -40.5), ('2006', -40.6), ('2013', -40.7), ('2007', -40.7), ('1963', -41.0), ('1955', -41.2), ('2003', -41.5), ('1969', -41.5), ('1996', -41.7), ('2010', -41.7), ('1962', -42.0), ('1951', -42.0), ('1950', -42.0), ('2011', -42.0), ('1968', -42.0), ('1982', -42.2), ('2002', -42.2), ('1976', -42.2), ('2014', -42.5), ('1977', -42.5), ('1998', -42.7), ('2012', -42.7), ('1958', -43.0), ('1985', -43.4), ('1959', -43.6), ('2001', -44.0), ('1965', -44.0), ('1981', -44.0), ('1979', -44.0), ('1986', -44.2), ('1971', -44.3), ('1956', -45.

In [3]:
# 2
date_temperature = lines.map(lambda x: (x[1][0:7], float(x[3])))
date_temperature = date_temperature.filter(lambda x: int(x[0][0:4])>=1950 and int(x[0][0:4])<=2014 and x[1]>=10)

count_month_occ = date_temperature.map(lambda x: (x[0], 1))
count_month_occ = count_month_occ.reduceByKey(lambda a,b:a+b)

print(count_month_occ.collect())

[('1963-04', 3126), ('2000-08', 110268), ('1982-04', 4542), ('1983-11', 726), ('1988-06', 64141), ('1990-09', 35374), ('1994-05', 22279), ('1995-09', 46831), ('1953-04', 2030), ('1962-03', 2), ('1960-01', 1), ('1952-02', 1), ('1975-05', 33103), ('2005-08', 114580), ('2010-05', 53027), ('2012-09', 71840), ('1981-05', 36033), ('1984-10', 19880), ('1987-04', 6647), ('2011-12', 479), ('1998-03', 1214), ('1950-04', 380), ('1968-11', 887), ('1955-02', 3), ('1957-08', 24829), ('1959-08', 23997), ('1967-03', 543), ('1972-03', 737), ('1978-10', 13057), ('2009-04', 23920), ('2012-10', 19135), ('1983-06', 50053), ('1987-09', 36363), ('1986-11', 1516), ('2008-02', 101), ('1983-12', 11), ('1991-02', 1), ('1966-04', 3172), ('1967-08', 55261), ('1974-08', 65873), ('1975-04', 6060), ('2001-05', 44819), ('2003-10', 9300), ('2007-08', 111030), ('1985-08', 56699), ('2001-04', 4853), ('1993-03', 366), ('1993-02', 33), ('1979-11', 44), ('2014-05', 58340), ('1966-07', 56062), ('2008-07', 127627), ('1986-06'

In [4]:
# distinct readings from each station
date_temperature = lines.map(lambda x: (x[1][0:7], (x[0],float(x[3]))))
date_temperature = date_temperature.filter(lambda x: int(x[0][0:4])>=1950 and int(x[0][0:4])<=2014 and x[1][1]>=10)

date_temperature = date_temperature.map(lambda x: (x[0], x[1][0])).distinct()

count_month_occ = date_temperature.map(lambda x: (x[0], 1))
count_month_occ = count_month_occ.reduceByKey(lambda a,b:a+b)

print(count_month_occ.collect())

[('1953-04', 106), ('1994-05', 299), ('1995-09', 315), ('2000-08', 325), ('1982-04', 252), ('1988-06', 322), ('1990-09', 312), ('1963-04', 286), ('1983-11', 174), ('1962-03', 2), ('1952-02', 1), ('1960-01', 1), ('2010-05', 319), ('2012-09', 306), ('2005-08', 306), ('1975-05', 367), ('1981-05', 337), ('1984-10', 324), ('1968-11', 88), ('1998-03', 146), ('1987-04', 263), ('1950-04', 37), ('2011-12', 66), ('1955-02', 3), ('1967-03', 171), ('1986-11', 151), ('1987-09', 323), ('2009-04', 276), ('2012-10', 246), ('1957-08', 130), ('1959-08', 125), ('1978-10', 341), ('1983-06', 337), ('1972-03', 188), ('1983-12', 4), ('2008-02', 27), ('1991-02', 1), ('1985-08', 314), ('1993-03', 108), ('1966-04', 311), ('1967-08', 358), ('1974-08', 372), ('1975-04', 274), ('2007-08', 302), ('2001-05', 316), ('2001-04', 235), ('2003-10', 204), ('1979-11', 30), ('1993-02', 20), ('1986-06', 314), ('1966-07', 352), ('1993-06', 289), ('1951-06', 110), ('2008-07', 314), ('2014-05', 296), ('1975-03', 37), ('1956-12'

In [5]:
# 3
date_temperature = lines.map(lambda x: (x[1], (x[0], float(x[3]))))
date_temperature = date_temperature.filter(lambda x: int(x[0][0:4])>=1960 and int(x[0][0:4])<=2014)

min_max_temperatures = date_temperature.reduceByKey(max) + date_temperature.reduceByKey(min)
month_temperatures = min_max_temperatures.map((lambda x: (x[0][0:7], (x[1][0], x[1][1]))))
sum_month_temperatures = month_temperatures.reduceByKey(lambda a,b: (a[0], a[1]+b[1]))
avg_month_temperatures = sum_month_temperatures.map((lambda x: (x[0], (x[1][0], round(x[1][1]/62, 1)))))

print(avg_month_temperatures.collect())

[('1982-04', ('99450', 1.3)), ('1985-12', ('99450', -7.4)), ('1990-09', ('99450', 8.0)), ('1994-02', ('99450', -8.3)), ('1995-09', ('99450', 9.5)), ('1960-01', ('99450', -5.4)), ('1962-03', ('99450', -7.2)), ('1963-04', ('99450', 0.9)), ('1983-11', ('99450', 0.5)), ('1988-06', ('99450', 10.2)), ('2000-08', ('99450', 13.7)), ('1994-05', ('99450', 5.0)), ('2012-01', ('99450', -2.9)), ('1962-12', ('99450', -5.7)), ('2010-05', ('99450', 7.9)), ('2011-12', ('99450', 0.6)), ('2012-09', ('99450', 9.5)), ('1981-05', ('99450', 5.5)), ('1987-01', ('99450', -13.4)), ('1998-03', ('99450', -1.9)), ('1968-11', ('99450', -2.5)), ('1984-10', ('99450', 6.4)), ('1987-04', ('99450', 0.2)), ('1975-05', ('99450', 6.4)), ('2005-08', ('99450', 15.2)), ('2009-04', ('99450', 4.1)), ('1983-06', ('99450', 9.3)), ('1983-12', ('99450', -2.0)), ('1986-11', ('99450', 2.9)), ('1987-09', ('99450', 6.8)), ('1972-03', ('99450', -1.7)), ('1978-10', ('99450', 4.9)), ('2012-10', ('99450', 5.0)), ('2007-02', ('99450', -4.0)

In [6]:
# Following code will save the result into /user/ACCOUNT_NAME/BDA/output folder
# max_temperatures.saveAsTextFile("output")